# Collects epochs for spikes

Add some explanation

#### Load modules

In [1]:
import h5py
import pickle
import time
import numpy as np
import pandas as pd

In [2]:
IDs = pickle.load(open('/home/maspe/filer/scripts/preprocessing/IDs.dict', 'rb'))
all_spikes = {mouse : [] for mouse in IDs['list_all']}
spikes_epochs = all_spikes


# Channels list
channels_list = ['ch01', 'ch02', 'ch03', 'ch04', 'ch05', 'ch06', 'ch07', 'ch08', 'ch09', 'ch10',
         'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch20',
         'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch30',
         'ch31', 'ch32']

# Epoching parameters
fs = 30000 ## Fs for spikes, not LFPs !!
n_secs = 3
window = n_secs * fs
entrances = dict()

#### Collecting spikes

In [3]:
for mouse in all_spikes.keys():
    print('Processing mouse {}...'.format(mouse))
    npys_dir = '/home/maspe/filer/SERT/' + mouse + '/npys/'
    spikes_dir = '/home/maspe/filer/SERT/' + mouse + '/spikes/results/'
    entrances_dir = '/home/maspe/filer/SERT/' + mouse + '/continuous/entradas.xlsx'  
    
    
    ### Collecting entrances times MOVER A SCRIPTS INICIALES!!
    print('Collecting entrances...')
    df = pd.read_excel(entrances_dir, sheet_name=0, header=None, names=["locs"])
    entrances[mouse] = np.array(df['locs'].tolist(), dtype='int') * 30 # Times 30 because of sampling at 30 KHz
    n_epochs = len(entrances[mouse])
    print('Number of entrances: {}'.format(n_epochs))
    
    # Creating epochs indexes
    epochs = []
    for entrance in entrances[mouse]:
        epochs.append((entrance - window, entrance + window))
        
    
    ### Loading spikes
    n_units = 0
    for channel in channels_list:
        results =  spikes_dir + channel + '.result.hdf5'
        fit     = h5py.File(results, 'r')
        
        
        for unit in fit['spiketimes'].keys():
            spikes    = fit['spiketimes'][unit][()]
            spikes_OF = spikes[spikes >= epochs[0][0]]
            
            if len(spikes_OF) == 0:
                print('Unit {} in channel {} has no spikes !!!'.format(unit, channel))
                all_spikes[mouse].append(0)
            else:
                all_spikes[mouse].append(spikes_OF)
            
            n_units += 1 
            
    print('Number of units: {}'.format(n_units))
    

    ### Epoching spikes
    print('Epoching spikes...')
    for unit in range(n_units):
    
        unit_epoch = []
        for epoch in epochs:
            unit_epoch.append(all_spikes[mouse][unit][(all_spikes[mouse][unit] >= epoch[0]) \
                                                      & (all_spikes[mouse][unit] <= epoch[1])] \
                                                         - epoch[0])
            
        spikes_epochs[mouse][unit] = unit_epoch

    print('Done!\n')
    
print('Saving dictionary...')
pickle.dump(spikes_epochs, open('/home/maspe/filer/SERT/ALL/npys/spikes2.epochs', 'wb'), protocol = 2)

print('All mice processed!')

Processing mouse SERT1984...
Number of entrances: 8
Number of units: 69
Epoching spikes...
Done!

Processing mouse SERT1985...
Number of entrances: 20
Number of units: 70
Epoching spikes...
Done!

Processing mouse SERT1659...
Number of entrances: 16
Number of units: 66
Epoching spikes...
Done!

Processing mouse SERT1678...
Number of entrances: 12
Number of units: 75
Epoching spikes...
Done!

Processing mouse SERT1908...
Number of entrances: 17
Number of units: 64
Epoching spikes...
Done!

Processing mouse SERT1668...
Number of entrances: 4
Number of units: 66
Epoching spikes...
Done!

Processing mouse SERT2018...
Number of entrances: 5
Number of units: 70
Epoching spikes...
Done!

Processing mouse SERT2013...
Number of entrances: 5
Number of units: 66
Epoching spikes...
Done!

Processing mouse SERT1597...
Number of entrances: 16
Number of units: 51
Epoching spikes...
Done!

Processing mouse SERT2024...
Number of entrances: 10
Number of units: 73
Epoching spikes...
Done!

Processing mou